<a href="https://colab.research.google.com/github/V4lciJr/Algoritmo-Apriori/blob/main/Algoritmo_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo Apriori
O Apriori é um algoritmo utilizado para realizarmos assossiação entre itens, muito ultilizado em mercados e marketplaces para fazer a associação entre produtos.

## Exemplo de uso
Digamos que você é um gerente ou diretor de uma grande rede de supermercados e deseja aumentar os lucros, para isso você quer descobrir um metódo de vendas onde a compra de um produto ele leva venda de outro. Sabendo disso, você tenta induzir seu cliente a comprar esses produtos ao mesmo tempo, ou seja, se o cliente compra pão, ele também compra café.
Esse algoritmo surgiu para podermos fazer essas associações de forma automatizada, abaixo iremos utilizar um pequeno dataset para exemplificarmos seu funcionamento.
Neste dataset nos atributos que possuem o seu valor como sim, significa que o cliente comprou aquele produto.

In [ ]:
import pandas as pd

#criando o dataset

columns = ['ID', 'Leite', 'Cafe', 'Cerveja', 'Pão', 'Manteiga', 'Arroz', 'Feijao']
dataset = [[1, 'não', 'sim', 'não', 'sim', 'sim', 'não', 'não'],
           [2, 'sim', 'não', 'sim', 'sim', 'sim', 'não', 'não'],
           [3, 'não', 'sim', 'não',	'sim', 'sim', 'não', 'não'],
           [4, 'sim', 'sim', 'não',	'sim', 'sim', 'não', 'não'],
           [5, 'não', 'não', 'sim',	'não', 'não', 'não', 'não'],
           [6, 'não', 'não', 'não',	'não', 'sim', 'não', 'não'],
           [7, 'não', 'não', 'não',	'sim', 'não', 'não', 'não'],
           [8, 'não', 'não', 'não',	'não', 'não', 'não', 'sim'],
           [9, 'não', 'não', 'não', 'não', 'não', 'sim', 'sim'],
           [10,'não', 'não', 'não',	'não', 'não', 'sim', 'não']]

df = pd.DataFrame(dataset, columns=columns)
df

,ID,Leite,Cafe,Cerveja,Pão,Manteiga,Arroz,Feijao
0,1,não,sim,não,sim,sim,não,não
1,2,sim,não,sim,sim,sim,não,não
2,3,não,sim,não,sim,sim,não,não
3,4,sim,sim,não,sim,sim,não,não
4,5,não,não,sim,não,não,não,não
5,6,não,não,não,não,sim,não,não
6,7,não,não,não,sim,não,não,não
7,8,não,não,não,não,não,não,sim
8,9,não,não,não,não,não,sim,sim
9,10,não,não,não,não,não,sim,não


# Criação da Classe Apriori
Esta é a classe onde implementaremos todo a lógica do algoritmo de estudo, este código foi inspirado na implementação de Bernardo Costa em seu github, com pequenas adaptações a realidade do nosso dataset, uma delas é o acréscimo de um parametro no construtor da classe e consequentemente no método transform_bool, onde de acordo com o que é passado, ele transforma o nosso dataset em True ou False

In [ ]:
from mlxtend.frequent_patterns import apriori


class Apriori:

    threshold = 0.5
    df = None

    def __init__(self, df, threshold=None, transform_bool=False, parametro=1):

        self._validade_df(df)

        self.df = df

        if threshold is not None:
            self.threshold = threshold

        if transform_bool:
            self._transform_bool(parametro)

    # método que valida se o dataframe é válido, caso não ele lança uma exceção
    def _validade_df(self, df=None):
        if df is None:
            raise Exception("DF inválido, deve ser dataFrame válido do pandas")


    # de acordo com o parametro recebido, transforma os valores para True ou False
    def _transform_bool(self, parametro):
        for column in self.df.columns:
            self.df[column] = self.df[column].apply(lambda x: True if x == parametro else False)


    # método que chama a função apriori da biblioteca importada
    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """
            define um sinalidor para usar o nome das colunas
            max_len: comprimento máximo dos conjuntos dos itens gerados
            count: conta os conjuntos dos itens
            retura um apriori dataFrame
        """

        apriori_df = apriori(
                     self.df,
                     min_support = self.threshold,
                     use_colnames = use_colnames,
                     max_len = max_len
        )


        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))

        return apriori_df

    def run(self, use_colnames=False, max_len=None, count=True):

        return self._apriori(
                use_colnames = use_colnames,
                max_len = max_len,
                count = count
        )

    def filter(self, apriori_df, length, threshold):

        if 'length' not in apriori_df.columns:
            raise Exception("Seu data não tem o parametro de comprimento. Execute-o com o parametro count = True")

        return apriori_df[(apriori_df['length'] == length) & (apriori_df['support'] >= threshold)]


In [ ]:
# executando o código

if 'ID' in df.columns: del df['ID']   # para este cálculo o ID não é relevante, então deletamos o ID do DataFrame

apriori_runner = Apriori(df, threshold=0.3, transform_bool=True, parametro='sim')
apriori_df = apriori_runner.run(use_colnames=True)
apriori_df

,support,itemsets,length
0,0.3,(Cafe),1
1,0.5,(Pão),1
2,0.5,(Manteiga),1
3,0.3,"(Pão, Cafe)",2
4,0.3,"(Cafe, Manteiga)",2
5,0.4,"(Pão, Manteiga)",2
6,0.3,"(Pão, Cafe, Manteiga)",3


In [ ]:
apriori_runner.filter(apriori_df, length=3, threshold=0.3)

,support,itemsets,length
6,0.3,"(Pão, Cafe, Manteiga)",3


# Resumo

Usamos o limite utilizado no livro, que é 0.3 e tamanho 2 para os conjuntos de associação, e conclúimos a associação de café e manteiga, mas se colocarmos o limite como 0.4 e mesmo tamanho de conjunto, temos pão e manteiga, o que concluísse que para esse caso, se aumentarmos o tamanho do conjunto para 3 manter o limite, teremos a associação entre pão, cafe e manteiga.

# Referências:

- Data Mining. Porto Alegre: Sagah, 2021.
- https://medium.com/@bernardo.costa/uma-introdu%C3%A7%C3%A3o-ao-algoritmo-apriori-60b11293aa5a
- https://github.com/obernardocosta/ml-demos/tree/master/201-apriori
- Dados: Data Mining. Porto Alegre: Sagah, 2021. (p. 260)